In [1]:
import tensorflow as tf
import ptb_data_model
import model
import numpy as np
import reader
import os

In [2]:
class Config(object):
    """config."""
    init_scale = 0.1
    learning_rate = 1.0
    max_grad_norm = 5
    num_layers = 2
    num_steps = 20
    hidden_size = 200
    max_epoch = 4
    max_max_epoch = 13
    keep_prob = 1.0
    lr_decay = 0.5
    batch_size = 20
    vocab_size = 10000

In [3]:
data_path = "/home/mor/projects/language_model/data"
save_path = "/home/mor/projects/language_model/save"

raw_data = reader.ptb_raw_data(data_path)
train_data, valid_data, test_data, _ = raw_data

config = Config()
eval_config = Config()
eval_config.batch_size = 1
eval_config.num_steps = 1

with tf.Graph().as_default():
    initializer = tf.random_uniform_initializer(-config.init_scale, config.init_scale)

    with tf.name_scope("Train"):
        train_input = ptb_data_model.PTBInput(config=config, data=train_data, name="TrainInput")
        with tf.variable_scope("Model", reuse=None, initializer=initializer):
            m = model.PTBModel(is_training=True, config=config, input_=train_input)
        tf.contrib.deprecated.scalar_summary("Training Loss", m.cost)
        tf.contrib.deprecated.scalar_summary("Learning Rate", m.lr)

    with tf.name_scope("Valid"):
        valid_input = ptb_data_model.PTBInput(config=config, data=valid_data, name="ValidInput")
        with tf.variable_scope("Model", reuse=True, initializer=initializer):
            mvalid = model.PTBModel(is_training=False, config=config, input_=valid_input)
        tf.contrib.deprecated.scalar_summary("Validation Loss", mvalid.cost)

    with tf.name_scope("Test"):
        test_input = ptb_data_model.PTBInput(config=eval_config, data=test_data, name="TestInput")
        with tf.variable_scope("Model", reuse=True, initializer=initializer):
            mtest = model.PTBModel(is_training=False, config=eval_config,input_=test_input)

    sv = tf.train.Supervisor(logdir=save_path)
    with sv.managed_session() as session:
        for i in range(config.max_max_epoch):
            lr_decay = config.lr_decay ** max(i + 1 - config.max_epoch, 0.0)
            m.assign_lr(session, config.learning_rate * lr_decay)

            print("Epoch: %d Learning rate: %.3f" % (i + 1, session.run(m.lr)))
            train_perplexity = model.run_epoch(session, m, eval_op=m.train_op,
                                                       verbose=True)
            print("Epoch: %d Train Perplexity: %.3f" % (i + 1, train_perplexity))
            valid_perplexity = model.run_epoch(session, mvalid)
            sv.saver.save(session, os.path.join('logs', "model.ckpt"), sv.global_step)
            from tensorflow.contrib.tensorboard.plugins import projector
            # Use the same LOG_DIR where you stored your checkpoint.
            summary_writer = tf.summary.FileWriter('logs')

            # Format: tensorflow/contrib/tensorboard/plugins/projector/projector_config.proto
            config1 = projector.ProjectorConfig()

            # You can add multiple embeddings. Here we add only one.
            embedding = config1.embeddings.add()
            embedding.tensor_name = m.embedding.name
            # Link this tensor to its metadata file (e.g. labels).
            embedding.metadata_path = os.path.join('logs', 'metadata.tsv')

            # Saves a configuration file that TensorBoard will read during startup.
            projector.visualize_embeddings(summary_writer, config1)
            
            print("Epoch: %d Valid Perplexity: %.3f" % (i + 1, valid_perplexity))

        test_perplexity = model.run_epoch(session, mtest)
        print("Test Perplexity: %.3f" % test_perplexity)

        if save_path:
            print("Saving model to %s." % save_path)
            sv.saver.save(session, save_path, global_step=sv.global_step)



Instructions for updating:
This op will be removed after the deprecation date. Please switch to tf.concat_v2().
Instructions for updating:
Please switch to tf.summary.FileWriter. The interface and behavior is the same; this is just a rename.
INFO:tensorflow:Model/global_step/sec: 0
Epoch: 1 Learning rate: 1.000
0.004 perplexity: 82.781 speed: 839 wps
0.104 perplexity: 61.447 speed: 1223 wps
INFO:tensorflow:Model/global_step/sec: 3.09975
0.204 perplexity: 67.317 speed: 1250 wps
0.304 perplexity: 65.541 speed: 1261 wps
INFO:tensorflow:Model/global_step/sec: 3.19769
0.404 perplexity: 65.541 speed: 1266 wps
INFO:tensorflow:Model/global_step/sec: 3.20381
0.504 perplexity: 65.296 speed: 1267 wps
0.604 perplexity: 64.336 speed: 1270 wps
INFO:tensorflow:Model/global_step/sec: 3.19931
0.703 perplexity: 64.539 speed: 1271 wps
0.803 perplexity: 65.030 speed: 1272 wps
INFO:tensorflow:Model/global_step/sec: 3.20058
0.903 perplexity: 67.615 speed: 1271 wps
INFO:tensorflow:Model/global_step/sec: 3.15

KeyboardInterrupt: 